# Day 12
Find the description of the problem [here](https://adventofcode.com/2024/day/12)!

## Part 1

Puzzle input:

In [9]:
with open("input_files/day_12.txt") as input_file:
    input = input_file.read()

Test input:

In [10]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """AAAA
# BBCD
# BBCC
# EEEC"""

Parse the input:

In [11]:
farm = [list(line) for line in input.split("\n")]

farm_width = len(farm[0])
farm_height = len(farm)

The approach I took to form the regions is to go through the farm left to right and top to bottom, and if the left or upper plot is of the same type, I add it to that region.

This sometimes separates plots that should be together, as they touch in another direction, so they are joined afterwards.

In [12]:
farm_regions = []

# Form quasi-regions by looking at the left and top plot. Some regions may be split
for y, line in enumerate(farm):
    for x, type in enumerate(line):
        if x == 5 and y == 2:
            pass
        for region in farm_regions:
            if (x - 1, y, type) in region or (x, y - 1, type) in region:
                region.append((x, y, type))
                break
        else:
            farm_regions.append([(x, y, type)])

# Convert quasi-regions to dictionary so that each one is indexed
farm_regions = {i: region for i, region in enumerate(farm_regions)}

# Get pairs of quasi-regions that together would form fully fledged regions
same_regions = set()
for i, region_1 in farm_regions.items():
    for j, region_2 in farm_regions.items():
        if region_1 == region_2:
            continue
        if region_1[0][2] == region_2[0][2]:
            for x, y, type in region_1:
                if (x - 1, y, type) in region_2 or (x + 1, y, type) in region_2 or (x, y - 1, type) in region_2 or (x, y + 1, type) in region_2:
                    same_regions.add(tuple(sorted((i, j))))
                    break

# Join quasi-regions pairs into quasi-regions groups
same_regions_reduced = []
for a, b in same_regions:
    overlapping_groups = []
    for group in same_regions_reduced:
        if a in group or b in group:
            overlapping_groups.append(group)
    if overlapping_groups:
        merged_group = [a, b]
        for group in overlapping_groups:
            merged_group.extend(group)
            same_regions_reduced.remove(group)
        same_regions_reduced.append(merged_group)
    else:
        same_regions_reduced.append([a, b])

# Remove duplicates in the quasi-regions groups
same_regions = []
for group in same_regions_reduced:
    same_regions.append(list(set(group)))

# Append the quasi-regions together
for region_ids in same_regions:
    main_id = region_ids[0]
    for id in region_ids[1:]:
        farm_regions[main_id].extend(farm_regions[id])
        farm_regions.pop(id)

from collections import defaultdict

# Calculate perimeters and areas of each region
perimeters = defaultdict(int)
areas = {}
for i, region in farm_regions.items():
    areas[i] = len(region)
    for x, y, type in region:
        if x == 0 or farm[y][x - 1] != type:
            perimeters[i] += 1
        if x == farm_width - 1 or farm[y][x + 1] != type:
            perimeters[i] += 1
        if y == 0 or farm[y - 1][x] != type:
            perimeters[i] += 1
        if y == farm_height - 1 or farm[y + 1][x] != type:
            perimeters[i] += 1

# Calculate the total price
price = 0
for area, perimeter in zip(areas.values(), perimeters.values()):
    price += area * perimeter

print(f"The total price of the fencing will be {price}.")

The total price of the fencing will be 1473276.


## Part 2

I read a little hint that said that the number of sides is always equal to the number of corners (unfortunately I didn't realize that myself ☹️).

To calculate the amount of corners, I took a two step approach, first the convex corners and then the concave ones.

The convex corners are easy, as they only depend on which sides of a given plot are neighboring another plot. The concave corners require information of two side plots.

In [13]:
# Format each region as a set of plots, each with a combination of corners {(x, y): (west, east, north, south)}
regions_with_fences = {}
for id, region in farm_regions.items():
    fences = {}
    for x, y, type in region:
        west = 0
        east = 0
        north = 0
        south = 0
        if not (x - 1, y, type) in region:
            west = 1
        if not (x + 1, y, type) in region:
            east = 1
        if not (x, y - 1, type) in region:
            north = 1
        if not (x, y + 1, type) in region:
            south = 1
        fences[(x, y)] = (west, east, north, south)
    regions_with_fences[id] = fences

# List all the cases of convex corners for each kind of plot
corner_cases = {
    (0, 0, 0, 0): 0,
    (0, 0, 0, 1): 0,
    (0, 0, 1, 0): 0,
    (0, 0, 1, 1): 0,
    (0, 1, 0, 0): 0,
    (0, 1, 0, 1): 1,
    (0, 1, 1, 0): 1,
    (0, 1, 1, 1): 2,
    (1, 0, 0, 0): 0,
    (1, 0, 0, 1): 1,
    (1, 0, 1, 0): 1,
    (1, 0, 1, 1): 2,
    (1, 1, 0, 0): 0,
    (1, 1, 0, 1): 2,
    (1, 1, 1, 0): 2,
    (1, 1, 1, 1): 4
}

# Calculate the number of corners
number_corners = {}
for id, region in regions_with_fences.items():
    count = 0
    for coords, corner in region.items():
        x, y = coords
        count += corner_cases[corner]
        # Take into account concave corners, which don't depend on a single plot
        if (x + 1, y) in region and region[(x + 1, y)][3] != 0 and (x, y + 1) in region and region[(x, y + 1)][1] != 0:
            count += 1
        if (x + 1, y) in region and region[(x + 1, y)][2] != 0 and (x, y - 1) in region and region[(x, y - 1)][1] != 0:
            count += 1
        if (x - 1, y) in region and region[(x - 1, y)][3] != 0 and (x, y + 1) in region and region[(x, y + 1)][0] != 0:
            count += 1
        if (x - 1, y) in region and region[(x - 1, y)][2] != 0 and (x, y - 1) in region and region[(x, y - 1)][0] != 0:
            count += 1
    number_corners[id] = count

# Calculate the total price
price = 0
for area, perimeter in zip(areas.values(), number_corners.values()):
    price += area * perimeter

print(f"The total price of the fencing will be {price}.")

The total price of the fencing will be 901100.
